# Описание проекта

Вы — аналитик крупного интернет-магазина. Вместе с отделом маркетинга вы подготовили список гипотез для увеличения выручки. Приоритизируйте гипотезы, запустите A/B-тест и проанализируйте результаты.

## Часть 1. Приоритизация гипотез. 

*В файле /datasets/hypothesis.csv 9 гипотез по увеличению выручки интернет-магазина с указанными параметрами Reach, Impact, Confidence, Effort.*

### Задача:

Примените фреймворк ICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.             
Примените фреймворк RICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.              
Укажите, как изменилась приоритизация гипотез при применении RICE вместо ICE. Объясните, почему так произошло.

## Часть 2. Анализ A/B-теста

*Вы провели A/B-тест и получили результаты, которые описаны в файлах /datasets/orders.csv и /datasets/visitors.csv.*

 ### Задача:

**Проанализируйте A/B-тест:**

* Постройте график кумулятивной выручки по группам. Сделайте выводы и предположения.  
* Постройте график кумулятивного среднего чека по группам. Сделайте выводы и предположения. 
* Постройте график относительного изменения кумулятивного среднего чека группы B к группе A. Сделайте выводы и предположения.  
* Постройте график кумулятивной конверсии по группам. Сделайте выводы и предположения.  \n",
* Постройте график относительного изменения кумулятивной конверсии группы B к группе A. Сделайте выводы и предположения.  
* Постройте точечный график количества заказов по пользователям. Сделайте выводы и предположения. 
* Посчитайте 95-й и 99-й перцентили количества заказов на пользователя. Выберите границу для определения аномальных пользователей. 
* Постройте точечный график стоимостей заказов. Сделайте выводы и предположения.  
* Посчитайте 95-й и 99-й перцентили стоимости заказов. Выберите границу для определения аномальных заказов.  
* Посчитайте статистическую значимость различий в конверсии между группами по «сырым» данным. Сделайте выводы и предположения. 
* Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «сырым» данным. Сделайте выводы и предположения.  
* Посчитайте статистическую значимость различий в конверсии между группами по «очищенным» данным. Сделайте выводы и предположения.  
* Посчитайте статистическую значимость различий в среднем чеке заказа между группами по «очищенным» данным. Сделайте выводы и предположения.

### Описание данных

***Данные для первой части***
Файл /datasets/hypothesis.csv. Скачать датасет              
Hypothesis — краткое описание гипотезы;       
Reach — охват пользователей по 10-балльной шкале;          
Impact — влияние на пользователей по 10-балльной шкале;    
Confidence — уверенность в гипотезе по 10-балльной шкале;                            
Efforts — затраты ресурсов на проверку гипотезы по 10-балльной шкале. Чем больше значение Efforts, тем дороже проверка гипотезы

***Данные для второй части***                      
Файл /datasets/orders.csv. Скачать датасет                
transactionId — идентификатор заказа;             
visitorId — идентификатор пользователя, совершившего заказ;               
date — дата, когда был совершён заказ;                    
revenue — выручка заказа;              
group — группа A/B-теста, в которую попал заказ.                  
Файл /datasets/visitors.csv. Скачать датасет           
date — дата;                
group — группа A/B-теста;                 
visitors — количество пользователей в указанную дату в указанной группе A/B-теста

### Шаг 1. Загрузка данных и подготовка их к анализу

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
from IPython.display import display

import scipy.stats as stats
import datetime as dt


In [2]:
hypothesis = pd.read_csv('/datasets/hypothesis.csv')
orders = pd.read_csv('/datasets/orders.csv')
visitors = pd.read_csv('/datasets/visitors.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/hypothesis.csv'

### Предобработка таблицы hypothesis - гипотезы

In [ ]:
hypothesis.info()

In [ ]:
hypothesis

*Данные целостные, корректные, типы правильные, дубликатов нет - таблица готова к анализу*

### Предобработка таблицы orders - заказы

In [ ]:
orders.info()

In [ ]:
orders.head()

In [ ]:
orders["date"] = orders["date"].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d')) # Поменяем тип столбца "date" на datetime

In [ ]:
orders.info()

In [ ]:
orders.describe()

In [ ]:
orders.duplicated().sum() # Проверим дубликаты

*Все данные корректны, целостны, дубликатов нет - и готовы к анализу. В процессе предобработки был поменян только тип столбца "date" на datetime*

### Предобработка таблицы visitors - посетители

In [ ]:
visitors.info()

In [ ]:
visitors.head()

In [ ]:
visitors["date"] = visitors["date"].map(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))

In [ ]:
visitors.info()

In [ ]:
visitors.describe()

In [ ]:
visitors.duplicated().sum()

*Все данные корректны, целостны, дубликатов нет - и готовы к анализу. В процессе предобработки был поменян только тип столбца "date" на datetime*

### Шаг 2. Приоритизация гипотез

- Примените фреймворк ICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
-  Примените фреймворк RICE для приоритизации гипотез. Отсортируйте их по убыванию приоритета.
-  Укажите, как изменилась приоритизация гипотез при применении RICE вместо ICE. Объясните, почему так произошло.

In [ ]:
pd.set_option('display.max_colwidth', 200)
hypothesis

In [ ]:
hypothesis['ICE'] = ((hypothesis['Impact'] * hypothesis['Confidence']) / hypothesis['Efforts']).round(1)
hypothesis['RICE'] = (hypothesis['Reach']  *hypothesis['Impact'] * hypothesis['Confidence']) / hypothesis['Efforts']

In [ ]:
pd.set_option('display.max_colwidth', 200)
display(hypothesis[['ICE', 'Hypothesis', 'Reach']].sort_values(by='ICE', ascending=False))

In [ ]:
pd.set_option('display.max_colwidth', 200)
hypothesis[['RICE', 'Hypothesis', 'Reach']].sort_values(by='RICE', ascending=False)

**Вывод:** Таблицы RICE и ICE по приоритизации различаются.Тройка лидеров у всех разная : в ICE это гипотезы с индексами 8, 0, 7 ,  в RICE - 7, 2, 0. Данное розичие продиктовано, тем что в фреймворке RICE мы учитываем такой параметр как- Reach (скольких пользователей затронет изменение, которое вы хотите внести), и в гипотезах 7 и 2 он очень высок (в 7 он = 10 из 10 А в 2 он =8 из 10, а в лидере ICE - в 8 он минимальный =1 из 10.  В итоге следует присмотреться к гипотезам 7 и 2.

# Шаг 3. Анализ A/B-теста

### 3.1 График кумулятивной выручки

In [ ]:
orders.head()

In [ ]:
visitors.head()

In [ ]:
datesGroups = orders[['date','group']].drop_duplicates()

ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])]. \
                                     agg({'date' : 'max', 
                                          'group' : 'max', 
                                          'transactionId' : pd.Series.nunique, 
                                          'visitorId' : pd.Series.nunique, 
                                          'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])]. \
                                       agg({'date' : 'max', 
                                            'group' : 'max', 
                                            'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData.head(5)

In [ ]:
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.figure(figsize=(15,5))
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='Кум.выручка группы A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='Кум.выручка группы B')
plt.grid()
plt.legend()
plt.title('График кумулятивной выручки групп "B" и "А"')
plt.show()

*По графику видно что выручка в группе "B" растёт чуть быстрее, чем в "A". 13 августа выручки примерно равны. А  после 13 августа в группе "B" выручка начинает расти значительно больше.Стоит отметить сильный всплеск выручки в группе "B" 19 августа, возможно был крупный заказ.*

Посмотрим какие заказы были в районе 19 августа

In [ ]:
cumulativeRevenueB.query(' "2019-08-17" <= date <= "2019-08-21" ')

In [ ]:
print('Медианное значение одного заказа 19 августа =',orders.query(' date == "2019-08-19" ')['revenue'].median())
orders.query(' date == "2019-08-19" ').sort_values(by='revenue', ascending=False).head(10)

*В таблице есть один заказ №425 на ~1,3млн в группе "B" 19 августа (в 492 раза больше медианы=2630 за этот день) в то время как остальные заказы не больше 44 тысяч . Поскольку этот заказ один, то его в принципе можно считать выбросом и при дальнейшем анализе это надо учитывать.*

### 3.2 График кумулятивного среднего чека

In [ ]:
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue']/cumulativeRevenueA['orders'], label='средний кумулятивный чек группы A')
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue']/cumulativeRevenueB['orders'], label='средний кумулятивный чек группы B')
plt.legend()
plt.title('График кумулятивного среденго чека групп "B" и "А"')
plt.show()

*Данный график похож с графиком кум.выручки. В первой половине в группе "B" чек преимущественно больше, но есть даты где и наоборот - чек больше в "A" - это 1 августа и 12-14 августа. Далее чек группы "B" резко начинает расти. Но также на графике видно сильное влияние на ср.чек огромного заказа №425, кстати после этого заказа (19 августа) средний чек в "B" постепенно начинает снижаться, в то время как в "A" он с 18 августа постепенно начинает расти.*

### 3.3 График относительного изменения кумулятивного среднего чека

In [ ]:
mergedCumulativeRevenue = cumulativeRevenueA.merge(cumulativeRevenueB, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])

plt.figure(figsize=(15,5))
plt.grid()
plt.plot(mergedCumulativeRevenue['date'], (mergedCumulativeRevenue['revenueB']/mergedCumulativeRevenue['ordersB'])/ \
         (mergedCumulativeRevenue['revenueA']/mergedCumulativeRevenue['ordersA'])-1)
plt.title('Относительное изменение кумулятивного среднего чека групп "B" к "А"')
plt.axhline(y=0, color='black', linestyle='--') 
plt.show();

*Средний чек "B" рос с 1 августа, 2 августа уже превысил "A" и рос до 7 августа. 7-8 начал падать до 13 августа, при этом 3 дня 12-14 акгуста средний чек "A" был больше "B". С 13 чек "B" опять начал расти и 15 августа  чек "B" стал больше и находился больше "A" до конца месяца. Опять же хорошо видно, что 19 августа был сильный всплеск за счет заказа №425 , а после средний чек "B" также начал снижаться.*

### 3.4 График кумулятивной конверсии

In [ ]:
cumulativeData['conversion'] = cumulativeData['orders']/cumulativeData['visitors']

cumulativeDataA = cumulativeData[cumulativeData['group']=='A']
cumulativeDataB = cumulativeData[cumulativeData['group']=='B']

plt.figure(figsize=(15,5))
plt.grid()
plt.plot(cumulativeDataA['date'], cumulativeDataA['conversion'], label='кумулятивная конверсия A')
plt.plot(cumulativeDataB['date'], cumulativeDataB['conversion'], label='кумулятивная конверсия B')
plt.title('Кумулятивная конверсия')
plt.legend()
plt.show();


*С 1-6 августа конверсия "A" была больше "В". Начиная с 6 августа и до конца месяца конверсия "В" преобладает над "А". При этом "В" в большей части находился в длиапазоне от 0.033-0.036, а "А"  от 0.031- 0.030 и ниже. Можно сделать вывод - конверсия группы "B" в целом значительно больше "A".*

### 3.5 График относительного изменения кумулятивной конверсии

In [ ]:
mergedCumulativeConversions = cumulativeDataA[['date','conversion']].merge(cumulativeDataB[['date','conversion']], \
                                                    left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
plt.figure(figsize=(15,5))
plt.grid()
plt.plot(mergedCumulativeConversions['date'], mergedCumulativeConversions['conversionB']/mergedCumulativeConversions['conversionA']-1, label="Относительный прирост конверсии группы B относительно группы A")

plt.title('Относительное изменение кумулятивной конверсии группы B к группе A')

plt.axhline(y=0, color='black', linestyle='--') 
plt.axhline(y=0.1, color='grey', linestyle='--')
plt.axhline(y=0.21, color='grey', linestyle='--')
plt.axhline(y=0.057, color='grey', linestyle='--')
plt.show();

*Данный график похож на график 3.4, то выводы будут схожи. С 1 по 5 августа - несколько дней когда конверсия в "A" была больше на 5-10%; но исключение в этот период это 2 августа - был  рост конверсии в группе "B" и по итогу дня результат "B" был на 5% больше "A". Начиная с 6 августа группа "B" лидировала до конца периода; её конверсия росла до 15 августа и разница с "A" на максимуме составила 21%, затем пошло постепенное снижение отностельной конверсии до 10%-14%.*

### 3.6 График количества заказов по пользователям

In [ ]:
ordersByUsers = orders.drop(['group', 'revenue', 'date'], axis=1).groupby('visitorId', as_index=False). \
                                                                        agg({'transactionId' : pd.Series.nunique})

ordersByUsers.columns = ['user_id','orders']

ordersByUsers.sort_values(by='orders',ascending=False).head(10)

In [ ]:
x_values = pd.Series(range(0,len(ordersByUsers)))

plt.figure(figsize=(15,5))
plt.grid()
plt.title('График количества заказов по пользователям')
plt.scatter(x_values, ordersByUsers['orders'], alpha=0.5) ;

*Больше всего пользователей совершили 1 заказ. Тех кто совершили 2 заказа уже намного меньше. И уже те кто совершил 3 заказа и более только уменьшается. Можно предположить что выше 3 заказов это аномалия.*

### 3.7 Граница заказов для определения аномальных пользователей

In [ ]:
print('Рассчитаем 95 и 99 перцентили по заказам : ', np.percentile(ordersByUsers['orders'], [95, 99])) 

In [ ]:
print('перцентили c 95 по 99 (по заказам) : ', np.percentile(ordersByUsers['orders'], [95,96,97,98, 99]))

*Большинство пользователей (98%) делают 1 или 2 заказа. Всё что выше - это аномалия*

### 3.8 График стоимостей заказов

In [ ]:
x_values = pd.Series(range(0,len(orders['revenue'])))
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График стоимостей заказов')
plt.scatter(x_values, orders['revenue'], alpha=0.5);

In [ ]:
norm_orders = orders[orders['revenue']<=195000]['revenue']
xn_values = pd.Series(range(0,len(norm_orders)))
plt.figure(figsize=(15,5))
plt.grid()
plt.title('График стоимостей заказов')
plt.scatter(xn_values, norm_orders, alpha=0.5);

*На графике виден тот самый аномальный заказ №425 (1.3 млн) и еще один на ~200 тысяц, остальные примерно не больше 100 тысяч.*

### 3.9 Граница для определения аномальных заказов (по выручке)

In [ ]:
print('Рассчитаем 95 и 99 перцентили по выручке заказов : ', np.percentile(orders['revenue'], [95, 99]))

*99% заказов меньше суммы 58233.2, установим ее в качестве границы выбросов*

### 3.10 Различия в конверсии между группами по «сырым» данным

In [ ]:
visitors.head()

In [ ]:
orders.head()

In [ ]:
# Соберем все данные по заказам и визитам (а также добавим кумулятивные показатели) в одну таблицу
visitorsADaily = visitors[visitors['group']=='A'][['date', 'visitors']]
visitorsADaily.columns = ['date', 'visitorsPerDateA']

visitorsACummulative = visitorsADaily.apply(lambda x: visitorsADaily[visitorsADaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateA' : 'sum'}), axis=1)
visitorsACummulative.columns = ['date', 'visitorsCummulativeA']

visitorsBDaily = visitors[visitors['group']=='B'][['date', 'visitors']]
visitorsBDaily.columns = ['date', 'visitorsPerDateB']

visitorsBCummulative = visitorsBDaily.apply(lambda x: visitorsBDaily[visitorsBDaily['date'] <= x['date']]. \
                                            agg({'date' : 'max', 'visitorsPerDateB' : 'sum'}), axis=1)
visitorsBCummulative.columns = ['date', 'visitorsCummulativeB']


ordersADaily = orders[orders['group']=='A'][['date', 'transactionId', 'visitorId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'transactionId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersADaily.columns = ['date', 'ordersPerDateA', 'revenuePerDateA']

ordersACummulative = ordersADaily.apply(
    lambda x: \
        ordersADaily[ordersADaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateA' : 'sum',
                'revenuePerDateA' : 'sum'}), axis=1).sort_values(by=['date'])
ordersACummulative.columns = ['date', 'ordersCummulativeA', 'revenueCummulativeA']

ordersBDaily = orders[orders['group']=='B'][['date', 'transactionId', 'visitorId', 'revenue']]\
    .groupby('date', as_index=False)\
    .agg({'transactionId' : pd.Series.nunique, 'revenue' : 'sum'})
ordersBDaily.columns = ['date', 'ordersPerDateB', 'revenuePerDateB']

ordersBCummulative = ordersBDaily.apply(
    lambda x: \
        ordersBDaily[ordersBDaily['date'] <= x['date']]\
            .agg({
                'date' : 'max',
                'ordersPerDateB' : 'sum',
                'revenuePerDateB' : 'sum'}), axis=1).sort_values(by=['date'])
ordersBCummulative.columns = ['date', 'ordersCummulativeB', 'revenueCummulativeB']

In [ ]:
data = ordersADaily.merge(ordersBDaily, left_on='date', right_on='date', how='left')\
    .merge(ordersACummulative, left_on='date', right_on='date', how='left')\
    .merge(ordersBCummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsADaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsBDaily, left_on='date', right_on='date', how='left')\
    .merge(visitorsACummulative, left_on='date', right_on='date', how='left')\
    .merge(visitorsBCummulative, left_on='date', right_on='date', how='left')
data.head()

**Применим критерий и отформатируем p-value, округлив его до трёх знаков после запятой. В sampleA сохраним выборку, где каждый элемент — число заказов определённого пользователя, в том числе 0. Значит, число элементов sampleA — это количество пользователей, сумма всех элементов — количество заказов. Чтобы получить конверсию в группе, поделим сумму заказов на число пользователей — найдём среднее в выборке sampleA методом mean(). Аналогично найдём конверсию группы B: SampleB.mean().
Выведем относительный прирост конверсии группы B: конверсия группы B / конверсия группы A - 1. Округлим до трёх знаков после запятой (пороговое значение возьмём равное 5%).**

**За нулевую гипотезу возьмём что: "статистически значимых различий в конверсии между группами нет"; тогда альтернативная гипотеза будет гласить - статистически значимые различия есть.**

In [ ]:
ordersByUsersA = orders[orders['group']=='A'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersA.columns = ['userId', 'orders']

ordersByUsersB = orders[orders['group']=='B'].groupby('visitorId', as_index=False).agg({'transactionId' : pd.Series.nunique})
ordersByUsersB.columns = ['userId', 'orders']

sampleA = pd.concat([ordersByUsersA['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), name='orders')],axis=0)

sampleB = pd.concat([ordersByUsersB['orders'],pd.Series(0, index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), name='orders')],axis=0)

print("alpha =",0.05)
print("p-value = {0:.3f}".format(stats.mannwhitneyu(sampleA, sampleB, alternative = 'two-sided')[1]))
print("Относительный показатель 'B' и 'A' = {0:.3f}".format(sampleB.mean()/sampleA.mean()-1)) 

*Вывод: p-value = 0.017 меньше 0.05. Значит, нулевую гипотезу о том, что статистически значимых различий в конверсии между группами нет, - отвергаем.Получается: по «сырым» данным различия в конверсии групп A и B есть. А относительный выигрыш группы "B" над "A" равен 13.8% .*

### 3.11 Статистическая значимость различий в среднем чеке заказа между группами по «сырым» данным

Чтобы рассчитать статистическую значимость различий в среднем чеке между группами, передадим критерию mannwhitneyu() данные о выручке с заказов (пороговое значение возьмём равное 5%). И найдём относительные различия в среднем чеке между группами.

За нулевую гипотезу возьмём что: "статистически значимых различий в средних чеках между группами нет"; тогда альтернативная гипотеза будет гласить - статистически значимые различия есть.

In [ ]:
print("alpha =",0.05)
print("p-value = {0:.3f}".format(stats.mannwhitneyu(orders[orders['group']=='A']['revenue'], orders[orders['group']=='B']['revenue'], alternative = 'two-sided')[1]))
print("Относительный показатель 'B' и 'A' = {0:.3f}".format(orders[orders['group']=='B']['revenue'].mean()/orders[orders['group']=='A']['revenue'].mean()-1)) 

*P-value значительно больше 0.05. Значит, мы не можем отвергать нулевую гипотезу. То есть средние чеки по "сырым" данным схожи. И при этом, средний чек группы B значительно выше среднего чека группы A (на 26%).*

### 3.12 Различия в конверсии между группами по «очищенным» данным

Вспомним - из предыдущих пунктов мы приняли за аномальных пользователей тех, кто совершил 3 и больше заказов или совершил заказ дороже 58233.2 рублей. Сделаем срезы пользователей с числом заказов больше 2 — usersWithManyOrders и пользователей, совершивших заказы дороже 58233.2 — usersWithExpensiveOrders. Объединим их в таблице abnormalUsers. Узнаем, сколько всего аномальных пользователей методом shape().

In [ ]:
usersWithManyOrders = pd.concat([ordersByUsersA[ordersByUsersA['orders'] > 2]['userId'], ordersByUsersB[ordersByUsersB['orders'] > 2]['userId']], axis = 0)
usersWithExpensiveOrders = orders[orders['revenue'] > 58233.2]['visitorId']
abnormalUsers = pd.concat([usersWithManyOrders, usersWithExpensiveOrders], axis = 0).drop_duplicates().sort_values()
print(abnormalUsers.head(5))
print("Всего 'ненормальных' пользователей =" ,len(abnormalUsers)) 

Узнаем, как их действия повлияли на результаты теста. Посчитаем статистическую значимость различий в конверсии между группами теста по очищенным данным. Сначала подготовим выборки количества заказов по пользователям по группам теста:

In [ ]:
sampleAFiltered = pd.concat([ordersByUsersA[np.logical_not(ordersByUsersA['userId'].isin(abnormalUsers))]['orders'], \
                             pd.Series(0, index=np.arange(data['visitorsPerDateA'].sum() - len(ordersByUsersA['orders'])), \
                                       name='orders')],axis=0)

sampleBFiltered = pd.concat([ordersByUsersB[np.logical_not(ordersByUsersB['userId'].isin(abnormalUsers))]['orders'], \
                             pd.Series(0, index=np.arange(data['visitorsPerDateB'].sum() - len(ordersByUsersB['orders'])), \
                                       name='orders')],axis=0) 


Применим статистический критерий Манна-Уитни к полученным выборкам,

За нулевую гипотезу возьмём что: "статистически значимых различий в конверсии между группами нет"; тогда альтернативная гипотеза будет гласить - статистически значимые различия есть.

In [ ]:
print("alpha =",0.05)
print("p-value = {0:.3f}".format(stats.mannwhitneyu(sampleAFiltered, sampleBFiltered, alternative = 'two-sided')[1]))
print("Относительный показатель 'B' и 'A' = {0:.3f}".format(sampleBFiltered.mean()/sampleAFiltered.mean()-1)) 

*Вывод: p-value = 0.01 меньше 0.05. Значит, нулевую гипотезу  - отвергаем. Это значит: что по «очищенным» данным различия в конверсии групп A и B есть. А относительный выигрыш группы "B" над "A" равен 17,4% (выше, чем с «сырыми» данными - 13.8%) .*

### 3.13 Статистическая значимость различий в среднем чеке заказа между группами по «очищенным» данным

А теперь п     осмотрим произошло ли что-нибудь с результатами по среднему чеку?

За нулевую гипотезу возьмём что: "статистически значимых различий в средних чеках между группами нет"; тогда альтернативная гипотеза будет гласить - статистически значимые различия есть.

In [ ]:
print("alpha =",0.05)
print("p-value = {0:.3f}".format(stats.mannwhitneyu(
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'],
    orders[np.logical_and(
        orders['group']=='B',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'], alternative = 'two-sided')[1]))

print("Относительный показатель 'B' и 'A' = {0:.3f}".format(
    orders[np.logical_and(orders['group']=='B',np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean()/
    orders[np.logical_and(
        orders['group']=='A',
        np.logical_not(orders['visitorId'].isin(abnormalUsers)))]['revenue'].mean() - 1)) 

*P-value значительно больше 0.05. Значит, причин отвергать нулевую гипотезу нет (как и с "Сырыми" данными). А вот средний чек группы B оказался незначительно (на 2%) ниже среднего чека группы A, в то время как по "сырым" данным, он был больше аж на ~26%, - и всё это из-за одного того самого аномального заказа №425 на 1,3 млн !*

### 3.14 Решение по результатам теста

## По результатам тестирования было выявлено:

- Есть различия в конверсии групп A и B есть по "сырым" и "очищенным" данным. Относительный выигрыш группы "B" над "A" равен 17,4% по "очищенным" и 13.8% по "сырым" данным.

- По "сырым" и "очищенным" данным различия в средних чеках групп A и B отстутствуют, несмотря на то, что средний чек группы B оказался незначительно (на 2%) ниже среднего чека группы A по "очищенным" данным, в то время как по "сырым" данным, он был больше аж на 26%, - и всё это из-за аномального заказа №425 на 1,3 млн. Всё логично аномальный заказ убрали и чеки сравнялись.

*В итоге можно принять решение остановить тестирование и констатировать победу группы B : группа B значительно лучше  группы A по конверсии, хотя по среднему чеку группы примерно равны.*